# Esempi di regressione logistica nell'analisi del linguaggio

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

## Sentiment analysis delle brevi recensioni

Proviamo ad usare Python per allenare un modello di regressione logistica per eseguire un task di **sentiment analysis** analologo a quello che abbiamo usato come esmpio a lezione.

## I dati

Prendiamo una piccola sezione del dataset contenuto nel corpus [Multiemo](https://clarin-pl.eu/dspace/handle/11321/798), un corpus multilingue contenente recensioni in 11 lingue in vari domini. Scegliamo i dati relativi al dominio *medico* in lingua italiana.

Questo è un esempio del tipo di testi conenuti:


>Super dottore e uomo da grande C . Grande esperienza e diagnosi accurate . Grande pazienza per gli anziani . Mi prendo cura della mia vecchia mamma da anni, e dico che siamo molto fortunati ad avere un medico del genere. Non so davvero cosa avremmo fatto se non fosse stato per il medico. Grazie a questo, mia madre è viva. Ogni visita ad uno specialista viene consultata con lui e penso che sia meglio di chiunque altro. Abbiamo una fiducia quasi illimitata in lui. Puoi fare molto di buono per il tuo medico ancora da scrivere. Purtroppo ha molti pazienti, è sovraccarico di lavoro (per questo temo anche per la sua salute) e l'accesso a lui è difficile ma sempre possibile. `__label__meta_plus_m`


La classificazione è a 5 categorie: positivi, negativi, neutri, ambigui. Scartiamo tutti i testi classificati come ambigui e neutri

## Preprocessing

La prima cosa che dobbiamo fare è trasformare i nostri testi in un vettore di *features* il cui impatto vogliamo misurare.

Prendiamo spunto dalla discussione in [Jurafsky e Martin](https://web.stanford.edu/~jurafsky/slp3/) e usiamo queste 6 variabili:

- nr. di parole positive
- nr. di parole negative
- contiene 'polarity shifters' (e.g. no, non, nessuno)? 1 se il testo ne contiene almeno una, altrimenti 0
- conto di pronomi di 1a o 2a persona
- punto esclamativo? 1 se il testo contiene il punto esclamativo; 0 in caso contrario
- log del numero totale di parole

Per effettuare la classificazione in parole positive e negative (e anche per ottenere la lista dei *polarity shifters*) ho utilizzato un [lessico di polarità dell'italiano](https://dspace-clarin-it.ilc.cnr.it/repository/xmlui/handle/20.500.11752/ILC-73) sviluppato dall'ILC-CNR.

Questo è il codice che ho utilizzato per trasformare ogni testo in un vettore di feature:

```python
def process_text(txt):
    doc = nlp(txt)
    positive_words = 0
    negative_words = 0
    hasShifters = 0
    pronouns = 0
    hasExclamations = 0
    log_nr = math.log(len(doc))

    for tok in doc:
        pol = polarity_entries.get(tok.lemma_, 'null')
        if pol == 'positive':
            positive_words += 1
        elif pol == 'negative':
            negative_words += 1
        
        if tok.lemma_ in polarity_shifters:
            hasShifters = 1

        if tok.lemma_ in ['io', 'tu', 'noi', 'voi']:
            pronouns += 1
        
        if tok.lower_ == '!':
            hasExclamations = 1
    return positive_words, negative_words, hasShifters, pronouns, hasExclamations, log_nr
```

Possiamo caricare il dataset (il testo delle recensioni non è riprodotto per comodità)

In [ ]:
df = pd.read_csv('../data/recensioni_feats.tsv', sep='\t', index_col=0)
df.head()

quante recensioni pos/neg ci sono?

In [ ]:
df.Label.value_counts()

### Alleniamo il modello

Usiamo la libreria `sckitlearn` per creare ed allenare il modello. La funzione per addestrare il modello richiede due argomenti obbligatori:
- una matrice che contiene la serie di features; immaginatela come una tabella con tante righe quante sono le nostre osservazioni e tante colonne quante sono le features
- un vettore con le classificazioni corrette

Cominciamo a creare l'input

In [ ]:
X = df[['PosWords', 'NegWords', 'hasShift', 'NrPron', 'hasExcl', 'logNr']].to_numpy()
y = df.Label.to_numpy()

Ora creiamo il modello. "Creare" in questo caso non vuol dire *allenare* sui dati. Per ora, vuol solamente dire impostare il funzionamento generale del modello, settando alcuni parametri iniziali. La classe di modelli `LogisticRegression` di `sklearn` ha molti parametri iniziali che possono essere regolarati, come si può vedere dalla sua documentazione:

In [ ]:
help(LogisticRegression)

Noi lavoriamo con i valori di default.

In [ ]:
model = LogisticRegression(solver='liblinear', random_state=0)

Ora possiamo addestrare ("fare il fit") dei dati!

In [ ]:
model.fit(X, y)

Ecco fatto! Possiamo vedere quali sono i pesi e quale è l'intercetto calcolato dal modello?

In [ ]:
model.intercept_

In [ ]:
model.coef_

Possiamo vedere le probabilità predette sui dati di training:

In [ ]:
model.predict_proba(X)[:10]

In [ ]:
model.predict(X)[:10]

### Valutazione

#### Riusiamo I dati di training

Facile, no? Adesso, però, è necessario **valutare** il nostro modello, ovvero sapere quale capacità predittiva abbia il nostro modello! Come facciamo?

Una risposta semplice semplice è quella di testare il nostro modello sugli stessi dati su cui si è addestrato

In [ ]:
model.score(X, y)

Uuuh... il nostro modello ha classificato correttamente solo il 67% dei testi di training... numeri non esattamente entusiasmanti!

Questo numero, peraltro non ci dice molto. Dove sbaglia il nostro classificatore? Quanti `positivi` classificati come `negativi` e viceversa abbiamo?

Per avere questo dato possiamo usare una cosiddetta `matrice di confusione` (confusion matrix), ovvero una tabella di errori di predizioni che ci restituisca il numero di:
- 0 classificati correttamente (veri negativi)
- 0 classificati erroneamente (falsi negativi)
- 1 classificati correttamente (veri positivi)
- 1 classificati erroneamente (falsi positivi)

In [ ]:
confusion_matrix(y, model.predict(X))

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
cm = confusion_matrix(y, model.predict(X))

fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(cm)
ax.xaxis.set(ticks=(0, 1), ticklabels=('Predicted 0s', 'Predicted 1s'))
ax.yaxis.set(ticks=(0, 1), ticklabels=('Actual 0s', 'Actual 1s'))
ax.set_ylim(1.5, -0.5)
for i in range(2):
    for j in range(2):
        ax.text(j, i, cm[i, j], ha='center', va='center', color='red')
plt.show()


In [ ]:
print(classification_report(y, model.predict(X)))

#### Usare un test set

Il corpus Multiemo, però, suddivide i dati in 3 parti: train, dev, test. La cosa più corretta da fare è usare i dati di test per valutare le performance del nostro modello. I dati sono già stati organizzati in features e label. Carichiamoli e vediamo!

In [ ]:
df_test = pd.read_csv('../data/recensioni_feats_TEST.tsv', sep='\t', index_col=0)
df_test.head()

In [ ]:
X_test = df_test[['PosWords', 'NegWords', 'hasShift', 'NrPron', 'hasExcl', 'logNr']].to_numpy()
y_test = df_test.Label.to_numpy()

In [ ]:
print(classification_report(y_test, model.predict(X_test)))

Vediamo ancora la matrice di confusione

In [ ]:
cm = confusion_matrix(y_test, model.predict(X_test))

fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(cm)
ax.xaxis.set(ticks=(0, 1), ticklabels=('Predicted 0s', 'Predicted 1s'))
ax.yaxis.set(ticks=(0, 1), ticklabels=('Actual 0s', 'Actual 1s'))
ax.set_ylim(1.5, -0.5)
for i in range(2):
    for j in range(2):
        ax.text(j, i, cm[i, j], ha='center', va='center', color='red')
plt.show()


Cosa succederebbe se predicessimo sempre la classe più frequente (ovvero: positivo, 0)? Azzeccheremmo comunque un buon numero di previsioni. Otterremmo, cioè, una accuratezza del 58%. È questo il numero che ci dà una stima più realistica del miglioramento approtato dal nostro modello.

In [ ]:
print(classification_report(y_test,  [0] * len(y_test) ))

Qui possiamo chiedere al modello di visualizzare le probabilità delle prime 10 osservazioni del test. Per ogni riga, il primo numero è la probabilità della classe 0, il secondo quella della classe 1.

Ricordatevi che abbiamo posto la soglia a 0.5: la classe che supera la soglia è quella assegnata dal nostro classificatore. Ad es., nel primo caso 0 ha una probabilità uguale a 0.68; per il nostro classificatore il primo testo sarà positivo!

In [ ]:
model.predict_proba(X_test)[:10]

Qui possiamo chiedere di darci in output le label che corrispondono alla probabilità:

In [ ]:
model.predict(X_test)[:10]

O addirittura possiamo stampare `positivo` e `negativo` come etichetta corrispondente a 0 e 1

In [ ]:
for res in model.predict(X_test)[:10]:
    if res == 0:
        print("positivo")
    else:
        print("negativo")

Che cos'è il **bias** $b$ che osserviamo nella proprietà del modello `model.intercept_` e che compare nella formula:

$z = \vec{w} \cdot \vec{x} + b$

Ricordiamo che $b$, in questa formula, equivale a:

In [ ]:
model.intercept_

Se avessimo un testo $x$ rappresentato da un vettore contenente solo una sequenza di 0 (ovvero: se tutte le feature del nostro testo fossero uguali a 0) otterremmo queste probabilità

In [ ]:
model.predict_proba([[0] * 6])

Ricordate che le probabilità (meglio, la probabilità $p$ della classe 1, dato che la probabilità della classe 0 è semplicemente 1-p) sono calcolate applicando la funzione logistica a $z$.

Dunque, che risultato otterremo se applicassimo una funzione ci permette di passare dalla probabilità della classe 1 quando tutte le features sono 0 al nostro coefficiente $z$?

La funzione che ci permette di passare dalle probabilità ai log-odds ($z$) è la cosidetta funzione *logit*:

$z = \log \left(\frac{p}{1-p}\right)$

Questo è il codice python che serve a calcolarla:

In [ ]:
import math

def logit(x):
    # le prima 2 righe di codice sono un buon modo per assicurarci che il valore passato 
    # sia una probabilità valida (tra 0 e 1)
    if x <= 0 or x >= 1:
        raise ValueError("Probability must be between 0 and 1, exclusive.")
    
    return math.log(x / (1 - x))

Proviamola con la probabilità della classe 1 quando tutti i valori delle feature sono 0 (ovvero $p=0.97116054$)

In [ ]:
logit(0.97116054)

Il nostro bias è:

In [ ]:
model.intercept_

Il bias non è nient'altro che il valore di $z$ quando il prodotto tra le feature ($\vec{x}$) e i pesi ($\vec{w}$) risulta 0

*CVD*!

Un aspetto utile della regressione logistica è che possiamo usare i pesi ($\vec{w}$) per farci un'idea delle variabili che influiscono di più sulla classificazione: l'idea (molto grossolana) è che più alto è il peso, più alta sarà l'influenza della feature in questione.

Possiamo (informalmente) utilizzare [queste formule](https://sefiks.com/2021/01/06/feature-importance-in-logistic-regression/) per avere un'idea dell'importanza relativa delle diverse features all'interno dell'equazione del modello. Ma una lettura attenta di un buon manuale di statistica per linguisti, come quello di [Gries](https://www.degruyter.com/document/doi/10.1515/9783110718256/html) citato più sotto. è fondamentale per chi volesse utilizzare i modelli lineari più seriamente per lo studio linguistico!

In [ ]:
feature_importance = pd.DataFrame(['PosWords', 'NegWords', 'hasShift', 'NrPron', 'hasExcl', 'logNr'], columns = ["feature"])
feature_importance["importance"] = pow(math.e, model.coef_[0])
feature_importance = feature_importance.sort_values(by = ["importance"], ascending=False)
ax = feature_importance.plot.barh(x='feature', y='importance')
plt.show()

## Studio Linguistico

Il libro *Statistics for Linguistics with R: A Practical Introduction* ([vedi qui](https://www.degruyter.com/document/doi/10.1515/9783110718256/html)) di S. Gries contiene una dettagliata e splendida spiegazione dell'uso della regressione logistica (insieme ad altri modelli) per lo studio linguistico.

Tra i vari dataset di esempio, ce n'è uno molto interessante (usato da Gries per illustrare la regressione logistica in contesto di studio quantitativo) relativo al posizionamento delle subordinate temporali e causali rispetto alla principale in un corpus misto inglese e tedesco. È riprodotto qui (dalla seconda ed. che io possiedo) nel file `data/clauseorder.csv`. 

Carichiamolo e diamogli un'occhiata

In [ ]:
df = pd.read_csv('../data/clauseorders.csv', sep='\t')
df.head()

Quello che noi vogliamo predire è l'ordine delle proposizioni, che qui è registrato nella colonna `ORDER`:

In [ ]:
df.ORDER.value_counts()

In [ ]:
feats = ['SUBORDTYPE', 'LEN_MC', 'LEN_SC', 'LENGTH_DIFF',
       'CONJ', 'MORETHAN2CL']
to_factorize = ['SUBORDTYPE', 'CONJ', 'MORETHAN2CL']

In [ ]:
for col in to_factorize:
    df[col] = df[col].factorize()[0]

df.head()

Qui *non* ripeteremo le analisi molto sofisticate che Gries discute nel suo libro. Ci limitiamo a giocare un po' con un modello creato con `sklearn`

In [ ]:
# trasformiamo le due categorie in 0 e 1: sc-mc diventa 0
y = df.ORDER.factorize()[0]

In [ ]:
model = LogisticRegression(solver='lbfgs', penalty='none', random_state=0)

In [ ]:
X = df[feats].values
model.fit(X, y)

In [ ]:
model.score(X, y)

In [ ]:
model.coef_

Possiamo (informalmente) utilizzare [queste formule](https://sefiks.com/2021/01/06/feature-importance-in-logistic-regression/) per avere un'idea dell'importanza relativa delle diverse features all'interno dell'equazione del modello. Ma una lettura attenta di Gries è fondamentale per chi volesse utilizzare i modelli lineari più seriamente per lo studio linguistico!

In [ ]:
feature_importance = pd.DataFrame(feats, columns = ["feature"])
feature_importance["importance"] = pow(math.e, model.coef_[0])
feature_importance = feature_importance.sort_values(by = ["importance"], ascending=False)
ax = feature_importance.plot.barh(x='feature', y='importance')
plt.show()

In [ ]:
feature_importance